#### 주의!!

이 실습은 가급적 NVIDIA GPU가 설치된 컴퓨터 환경이거나 Google Colab에서 진행해주세요.

## 환경 준비 
(Google Colab 환경에서 사용하세요)

In [ ]:
!pip install transformers
!pip install accelerate

## 데이터 다운로드
(Google Colab 환경에서 사용하세요)


데이터는 재량으로 huggingface hub를 이용하여 받으셔도 됩니다.

In [ ]:
!mkdir -p data_in/KOR/naver_movie
!wget https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_train.txt \
              -O data_in/KOR/naver_movie/ratings_train.txt
!wget https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_test.txt \
              -O data_in/KOR/naver_movie/ratings_test.txt

In [ ]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer

import torch
import pandas as pd
import numpy as np
import re

import random
from random import sample

from tqdm import tqdm

In [ ]:
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
SEED_NUM = 1234
np.random.seed(SEED_NUM)
random.seed(SEED_NUM)

## Alpaca-polyglot nsmc 테스트

### Alpaca-polyglot 로드

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-5.8b")

In [ ]:
cls_model = AutoModelForCausalLM.from_pretrained("beomi/KoAlpaca-Polyglot",
                                                 torch_dtype=torch.float16,
                                                 low_cpu_mem_usage=True).cuda()

In [ ]:
cls_model.config.max_length = 2048
cls_model.config.pad_token_id = 0

### 퓨샷 러닝을 위한 네이버 영화 리뷰 데이터 EDA 및 구성

In [ ]:
# 데이터 전처리 준비
DATA_IN_PATH = './data_in/KOR'
DATA_OUT_PATH = './data_out/KOR'

DATA_TRAIN_PATH = os.path.join(DATA_IN_PATH, 'naver_movie', 'ratings_train.txt')
DATA_TEST_PATH = os.path.join(DATA_IN_PATH, 'naver_movie', 'ratings_test.txt')

train_data = pd.read_csv(DATA_TRAIN_PATH, header = 0, delimiter = '\t', quoting = 3)
train_data = train_data.dropna()

In [ ]:
print('데이터 positive 라벨: ', '긍정')
print('데이터 negative 라벨: ', '부정')

In [ ]:
print('학습 예시 케이스 구조: ', '문장: 오늘 기분이 좋아\n감정: 긍정\n')

In [ ]:
print('gpt 최대 토큰 길이: ', cls_model.config.max_position_embeddings)

In [ ]:
sent_lens = [len(tokenizer(s).input_ids) for s in tqdm(train_data['document'])]

print('Few shot 케이스 토큰 평균 길이: ', np.mean(sent_lens))
print('Few shot 케이스 토큰 최대 길이: ', np.max(sent_lens))
print('Few shot 케이스 토큰 길이 표준편차: ',np.std(sent_lens))
print('Few shot 케이스 토큰 길이 80 퍼센타일: ',np.percentile(sent_lens, 80))

In [ ]:
train_fewshot_data = []

for train_sent, train_label in tqdm(train_data[['document', 'label']].values):
    tokens = tokenizer(train_sent).input_ids

    if len(tokens) <= 25:
        train_fewshot_data.append((train_sent, train_label))

### 네이버 영화 리뷰 데이터를 활용한 퓨샷 러닝 및 평가

In [ ]:
test_data = pd.read_csv(DATA_TEST_PATH, header=0, delimiter='\t', quoting=3)
test_data = test_data.dropna()
test_data.head()

In [ ]:
# Full Dataset
# sample_size = len(test_data)

# Sampled Dataset
sample_size = 500

train_fewshot_samples = []

for _ in range(sample_size):
    fewshot_examples = sample(train_fewshot_data, 10)
    train_fewshot_samples.append(fewshot_examples)

if sample_size < len(test_data['id']):
    test_data = test_data.sample(sample_size, random_state=SEED_NUM)

### koalpaca 프롬프트 1 방식 실험

In [ ]:
def build_prompt_text(sent):
    return "문장: " + sent + '\n감정:'

def clean_text(sent):
    sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", sent)
    return sent_clean

real_labels = []
pred_tokens = []

total_len = len(test_data[['document','label']].values)

for i, (test_sent, test_label) in tqdm(enumerate(test_data[['document','label']].values), total=total_len):
    prompt_text = ''

    for ex in train_fewshot_samples[i]:
        example_text, example_label = ex
        cleaned_example_text = clean_text(example_text)
        appended_prompt_example_text = build_prompt_text(cleaned_example_text)
        appended_prompt_example_text += ' 긍정\n' if example_label == 1 else ' 부정\n'
        prompt_text += appended_prompt_example_text

    cleaned_sent = clean_text(test_sent)
    appended_prompt_sent = build_prompt_text(cleaned_sent)

    prompt_text += appended_prompt_sent

    tokens = tokenizer(prompt_text, return_tensors="pt")
    token_ids, attn_mask = tokens.input_ids.cuda(), tokens.attention_mask.cuda()
    gen_tokens = cls_model.generate(input_ids=token_ids, attention_mask=attn_mask,
                                    max_new_tokens=1, pad_token_id=0)
    pred = tokenizer.batch_decode(gen_tokens[:, -1])[0].strip()

    pred_tokens.append(pred)
    real_labels.append('긍정' if test_label == 1 else '부정')

In [ ]:
accuracy_match = [p == t for p, t in zip(pred_tokens, real_labels)]
accuracy = len([m for m in accuracy_match if m]) / len(real_labels)

print(accuracy)

### koalpaca 프롬프트 2 방식 실험

In [ ]:
def build_prompt_text(sent):
    return '다음 문장은 긍정일까요 부정일까요?\n' + sent + '\n정답:'

real_labels = []
pred_tokens = []


real_labels = []
pred_tokens = []

total_len = len(test_data[['document','label']].values)

for i, (test_sent, test_label) in tqdm(enumerate(test_data[['document','label']].values), total=total_len):
    prompt_text = ''

    for ex in train_fewshot_samples[i]:
        example_text, example_label = ex
        cleaned_example_text = clean_text(example_text)
        appended_prompt_example_text = build_prompt_text(cleaned_example_text)
        appended_prompt_example_text += ' 긍정\n' if example_label == 1 else ' 부정\n'
        prompt_text += appended_prompt_example_text

    cleaned_sent = clean_text(test_sent)
    appended_prompt_sent = build_prompt_text(cleaned_sent)

    prompt_text += appended_prompt_sent

    tokens = tokenizer(prompt_text, return_tensors="pt")
    token_ids, attn_mask = tokens.input_ids.cuda(), tokens.attention_mask.cuda()
    gen_tokens = cls_model.generate(input_ids=token_ids, attention_mask=attn_mask,
                                    max_new_tokens=1, pad_token_id=0)
    pred = tokenizer.batch_decode(gen_tokens[:, -1])[0].strip()

    pred_tokens.append(pred)
    real_labels.append('긍정' if test_label == 1 else '부정')

In [ ]:
accuracy_match = [p == t for p, t in zip(pred_tokens, real_labels)]
accuracy = len([m for m in accuracy_match if m]) / len(real_labels)

print(accuracy)

## polyglot 5.8b nsmc 테스트

### polyglot 로드

colab에서는 유료버전만 실행 가능합니다.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-5.8b")

In [ ]:
cls_model = AutoModelForCausalLM.from_pretrained("EleutherAI/polyglot-ko-5.8b",
                                                 torch_dtype=torch.float16,
                                                 low_cpu_mem_usage=True).cuda()

### polyglot 프롬프트 1 방식 실험

In [ ]:
def build_prompt_text(sent):
    return "문장: " + sent + '\n감정:'

def clean_text(sent):
    sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", sent)
    return sent_clean

real_labels = []
pred_tokens = []

total_len = len(test_data[['document','label']].values)

for i, (test_sent, test_label) in tqdm(enumerate(test_data[['document','label']].values), total=total_len):
    prompt_text = ''

    for ex in train_fewshot_samples[i]:
        example_text, example_label = ex
        cleaned_example_text = clean_text(example_text)
        appended_prompt_example_text = build_prompt_text(cleaned_example_text)
        appended_prompt_example_text += ' 긍정\n' if example_label == 1 else ' 부정\n'
        prompt_text += appended_prompt_example_text

    cleaned_sent = clean_text(test_sent)
    appended_prompt_sent = build_prompt_text(cleaned_sent)

    prompt_text += appended_prompt_sent

    tokens = tokenizer(prompt_text, return_tensors="pt")
    token_ids, attn_mask = tokens.input_ids.cuda(), tokens.attention_mask.cuda()
    gen_tokens = cls_model.generate(input_ids=token_ids, attention_mask=attn_mask,
                                    max_new_tokens=1, pad_token_id=0)
    pred = tokenizer.batch_decode(gen_tokens[:, -1])[0].strip()

    pred_tokens.append(pred)
    real_labels.append('긍정' if test_label == 1 else '부정')

In [ ]:
accuracy_match = [p == t for p, t in zip(pred_tokens, real_labels)]
accuracy = len([m for m in accuracy_match if m]) / len(real_labels)

print(accuracy)

### polyglot 프롬프트 2 방식 실험

In [ ]:
def build_prompt_text(sent):
    return '다음 문장은 긍정일까요 부정일까요?\n' + sent + '\n정답:'

real_labels = []
pred_tokens = []


real_labels = []
pred_tokens = []

total_len = len(test_data[['document','label']].values)

for i, (test_sent, test_label) in tqdm(enumerate(test_data[['document','label']].values), total=total_len):
    prompt_text = ''

    for ex in train_fewshot_samples[i]:
        example_text, example_label = ex
        cleaned_example_text = clean_text(example_text)
        appended_prompt_example_text = build_prompt_text(cleaned_example_text)
        appended_prompt_example_text += ' 긍정\n' if example_label == 1 else ' 부정\n'
        prompt_text += appended_prompt_example_text

    cleaned_sent = clean_text(test_sent)
    appended_prompt_sent = build_prompt_text(cleaned_sent)

    prompt_text += appended_prompt_sent

    tokens = tokenizer(prompt_text, return_tensors="pt")
    token_ids, attn_mask = tokens.input_ids.cuda(), tokens.attention_mask.cuda()
    gen_tokens = cls_model.generate(input_ids=token_ids, attention_mask=attn_mask,
                                    max_new_tokens=1, pad_token_id=0)
    pred = tokenizer.batch_decode(gen_tokens[:, -1])[0].strip()

    pred_tokens.append(pred)
    real_labels.append('긍정' if test_label == 1 else '부정')

In [ ]:
accuracy_match = [p == t for p, t in zip(pred_tokens, real_labels)]
accuracy = len([m for m in accuracy_match if m]) / len(real_labels)

print(accuracy)